In [1]:
from math import pi

import csv
import numpy as np
import matplotlib.pyplot as plt


from model_Trains import Trains
from model_Aproximators import Aproximators
from model_DataGenerator import DataGenerator


from Base import *
from testTrains import testTrains

from model_Trace import Trace

import os
import csv

In [2]:
epsU = 1
epsZR = 0.025
epsZO = pi/2

steps = 10
length = 1200

In [3]:
trains = [0,1,2,3]
dop = [0 for i in trains]
maxEs = [0 for i in trains]
maxSKOs = [0 for i in trains]
SKOs = [0 for i in trains]
SKOs_S = [0 for i in trains]
times = [0 for i in trains]




for i in range(steps):
    orig, E2, E3, time = run(
        testTrains(1/10),
        DataGenerator["Generators"][1](
            epsU = epsU,
            epsZR = epsZR,
            epsZO = epsZO,            
            seed = i
        ),
        steps = length
    )
    for i, q in enumerate(zip(E2, time)):
        e2, t = q
        SKO = sum(e2)/len(e2)
        
        dop[i] += 1 if SKO < sum(E2[0])/len(E2[0]) else 0
        
        maxEs[i] = max(maxEs[i], max(e2))
        maxSKOs[i] = max(maxSKOs[i], SKO)
        
        SKOs[i] += SKO
        SKOs_S[i] += SKO/orig[2]
        times[i] += sum(t)/len(t)
        
for i in range(len(maxEs)):
    SKOs[i] /= steps
    SKOs_S[i] /= steps
    times[i] /= steps

In [6]:
maxEs, maxSKOs, SKOs, SKOs_S, times
with open("table_result.csv" , "w", encoding='cp1251',newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=';', dialect=csv.excel_tab)
    
    lines = [
        ["Алгоритм", "Ориг", "Фурье", "Уск", "Тряск"],
        ["Лучше оригинального"] + [str(d).replace(".",",") for d in dop],
        ["Максимальная ошибка, м"] + [str(d).replace(".",",") for d in maxEs],
        ["Максимальное СКО, м"] + [str(d).replace(".",",") for d in maxSKOs],
        ["Среднее СКО, м"] + [str(d).replace(".",",") for d in SKOs],
        ["Среднее отношение СКО/S"] + [str(d).replace(".",",") for d in SKOs_S],
        ["Среднее время итерации, мc"] + [str(d).replace(".",",") for d in times]
    ]
    
    for line in lines:
        writer.writerow(line)